# Parallelism Benchmarks
This notebook illustrates the effect of enabling scheduler parallel execution across backends.

In [ ]:
from quasar import Circuit, Scheduler
from quasar.cost import Backend
from quasar.planner import Planner
import time

circuit = Circuit([
    {"gate": "T", "qubits": [0]},
    {"gate": "T", "qubits": [1]},
    {"gate": "H", "qubits": [0]},
    {"gate": "H", "qubits": [1]},
])

class SleepBackend:
    def load(self, n):
        pass
    def apply_gate(self, gate, qubits, params):
        time.sleep(0.05)
    def extract_ssd(self):
        return None

scheduler_p = Scheduler(
    backends={Backend.DECISION_DIAGRAM: SleepBackend()},
    planner=Planner(),
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
    parallel_backends=[Backend.DECISION_DIAGRAM],
)
start = time.time(); scheduler_p.run(circuit); par = time.time() - start

scheduler_s = Scheduler(
    backends={Backend.DECISION_DIAGRAM: SleepBackend()},
    planner=Planner(),
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
    parallel_backends=[],
)
start = time.time(); scheduler_s.run(circuit); ser = time.time() - start
print(f"parallel {par:.3f}s vs serial {ser:.3f}s")


In [ ]:

# Record parameters and results
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'notebook'

# Collect simple parameters from globals
_params = {
    k: v for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))
}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f"../results/{nb_name}_params.json", 'w') as f:
    json.dump(_params, f, indent=2)
if 'results' in globals():
    try:
        with open(f"../results/{nb_name}_results.json", 'w') as f:
            json.dump(results, f, indent=2)
    except TypeError:
        pass
print(json.dumps(_params, indent=2))
